In [ ]:
import os
from langchain_core.tools import tool
from langgraph.graph import MessagesState, START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
import finnhub


In [ ]:
#Initialize Finnhub API client
finnhub_client = finnhub.client(api_key=os.getenv('FINNHUB_API_KEY'))


#Define the tool : querying stock prices using the Finnhub API
@tool
def get_stock_price(symbol: str):
    """ Retrieve the latest stock price for the given symbol """
    quote = finnhub_client.quote(symbol)
    return f"The current price for  {symbol} is ${quote}"



#Define the tool for purchasing stock 
@tool
def purchase_stock(symbol: str, quantity: int):
    """Simulate stock purchase and return a confirmation messages"""
    return f"Purchased {quantity} shares of {symbol} at the current market"


#Register the tool in the tool node
tools = [get_stock_price, purchase_stock]

tool_node = ToolNode(tools)    

#Set up the AI model
model = ChatOpenAI(model="gpt-40-mini")
model = model.bind_tools(tools)


#Define the function that simulate reasoning and invokes the model
def agent_reasoning(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}


#Define conditional logic to determine whether to continue or  stop
def shoul_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    #If there are no tool calls finish the process
    if not last_message.tool_calls:
        return "end"
    return "continue" 


#Build the ReAct agent using LangGraph
workflow = StateGraph(MessagesState)
#Add nodes agent reasoning and tool invocation (stock price retrieval)
workflow.add_node("agent_reasoning", agent_reasoning)
workflow.add_node("call_tool", tool_node)

#Define the flow
workflow.add_edge(START, "agent_reasoning")
#Conditional edges continue to toll call or end the process
workflow.add_conditional_edges("agent_reasoning",
shoul_continue,{
    "continue": "call_tool",
    "end": END
})

#Normal edge after invoking the tool return to agent reasoning 
workflow.add_edge("call_tool", "agent_reasoning")
#Set up memory for breakpoints 
memory = MemorySaver()
app = workflow.compile(checkpointer=memory, interrupt_after=["call_tool"])



In [ ]:
#Simulate user input for stock symbol
initial_user = {"messages": [{"role": "user", "content": "Should i buy AAPL stock today?"}]}
thread = {"configurable": {"thread_id": "1"}}

#Run the agent reasoning step first
for event in app.stream(initial_user,thread,stream_mode="values"):
    print(event)


#Pausing  for human approval before retrieving stock price
user_approval = input("Do you approve querying the stock price for AAPL? (yes/no)")
if user_approval.lower() == "yes":
    #Continue with tool invocation to get stock price
    for event in app.stream(None,thread,stream_mode="values"):
        print(event)

else:
    print("Execution halted by user.")            
